# Predicting Ethiopian Vehicle Insurance Premiums

- The goal of this project is to clean, analyse and predict vehicle insurance premiums of the state-owned Ethiopian Insurance Corporation (one of the biggest insurance companies in Ethiopia).
- The dataset we'll use describes vehicles, their insurance premiums and other insurance related atributes from July 2011 to June 2018. It can be found on [Mendeley Data](https://data.mendeley.com/datasets/34nfrk36dt/1).



#### Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon, kstest
import sklearn as sk
from sklearn.linear_model import LinearRegression
from fuzzywuzzy import process

import requests
from bs4 import BeautifulSoup

#%config InlineBackend.figure_formats = ['svg']

insurance_data_1 = pd.read_csv("insuranceData/motor_data11-14lats.csv")
insurance_data_2 = pd.read_csv("insuranceData/motor_data14-2018.csv")

### Predefined function 

In [ ]:
def plot_counts_and_premiums(ax, count_series, premium_series, xlabel):
    index = count_series.index
    x = np.arange(len(index))
    w = 0.4

    ax.bar(x - w/2, count_series.values, w, label='Policy Count', color='tab:red')
    ax.set_ylabel('Policy Count', color='tab:red')
    ax.tick_params(axis='y', labelcolor='tab:red')
    ax.set_xlabel(xlabel)
    ax.set_xticks(x)
    ax.set_xticklabels(index)

    ax2 = ax.twinx()
    ax2.bar(x + w/2, premium_series.loc[index].values, w, label='Premium Sum', color='tab:blue')
    ax2.set_ylabel('Premium Sum', color='tab:blue')
    ax2.tick_params(axis='y', labelcolor='tab:blue')

## Dataset overview

Both of the provided dataset files include the same entry attributes and differ only in entry dates. They will need to be merged.

In [ ]:
insurance_data = pd.concat([insurance_data_1, insurance_data_2], ignore_index=True)

print(insurance_data.shape)

We have 802036 insurance policy records and 16 policy related attributes.

Now let's look whether the provided attributes have been read correctly.

In [ ]:
insurance_data.head(10)

There are multiple entries regarding the same vehicle as it has to be reinsured every year. That can lead to up to 7 entries for the same vehicle with only the premium amout fluctuating.

In [ ]:
insurance_data.info()

Columns seem to have been read correctly. Let's now look at the values in individual columns.

### Sex

In [ ]:
insurance_data["SEX"].value_counts()

In the dataset, there are 3 unique sex values with 0 being legal entities, 1 - males and 2 - females. The number of insurance contracts in which men are the policyholders is 4.67 times greater that the number of contracts with female policyholders. That is due to women being less likely to have a drivers license in Ethiopia.

Let's remap the value for better clarity.

In [ ]:
sex_mapping = {0:'LEGAL ENTITY', 1:'MALE', 2:'FEMALE'}

insurance_data["SEX"] = insurance_data["SEX"].map(sex_mapping)

insurance_data["SEX"].value_counts()

### Insurance start & end date

In [ ]:
insurance_data["INSR_BEGIN"].value_counts()

In [ ]:
insurance_data["INSR_END"].value_counts()

There do not seem to be any obvious errors in the data. Yet, we must check whether there are entries where the insurance start date is later than the end date.

In [ ]:
insurance_data["INSR_BEGIN"] = pd.to_datetime(insurance_data["INSR_BEGIN"], format="%d-%b-%y")

insurance_data["INSR_END"] = pd.to_datetime(insurance_data["INSR_END"], format="%d-%b-%y")

end_greater_start = insurance_data["INSR_BEGIN"] > insurance_data["INSR_END"]
length = len(end_greater_start[end_greater_start == True])
length

No end values are earlier than start values. We can now visualize the variable data.

In [ ]:
insurance_data["INSR_START_MONTH"] = pd.to_datetime(insurance_data["INSR_BEGIN"], format="%d-%b-%y").dt.month
insurance_data["INSR_START_YEAR"] = pd.to_datetime(insurance_data["INSR_BEGIN"], format="%d-%b-%y").dt.year

start_months = insurance_data["INSR_START_MONTH"].value_counts().sort_index()
start_months_premium = insurance_data.groupby("INSR_START_MONTH")["PREMIUM"].sum()

start_years = insurance_data["INSR_START_YEAR"].value_counts().sort_index()
start_years_premium = insurance_data.groupby("INSR_START_YEAR")["PREMIUM"].sum()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

plot_counts_and_premiums(axes[0], start_months, start_months_premium, 'Month')
plot_counts_and_premiums(axes[1], start_years, start_years_premium, 'Year')

fig.tight_layout()
plt.show()

From the charts it becomes clear that policy count closely correlates with premium sums with regard to both the month and the year of the insurance start date. One month, July, stand out as the month in which the most policies are introduced. In terms of the trend regarding the year, there is a clear pattern of growth from 2011 to 2017 and a sharp drop off of new policies in 2018. The drop off can be explained by the end of data collection period being June of 2018. We can check whether growth in the month of June in the respective years is equally as rapid.

In [ ]:
before_june = insurance_data[(insurance_data["INSR_START_YEAR"].isin([2017, 2018])) & (insurance_data["INSR_START_MONTH"] < 7)]

before_june.groupby("INSR_START_YEAR").size().reset_index(name='Policy Count')

The pace of growth is about the same.

Let's also create a dummy variable for insurance start days.

In [ ]:
insurance_data["INSR_START_DAY"] = pd.to_datetime(insurance_data["INSR_BEGIN"], format="%d-%b-%y").dt.day

### Effective year

In [ ]:
insurance_data["EFFECTIVE_YR"].value_counts()

The effective year variable indicates what year the policy came into effect (was first insured with the company). There are numerous records that indicate a year before the historic start date of the dataset (2011).

Yet, the column contains values that are not indicative of a number and should be removed. Since there are a total of 802036 records, we can afford to lose quite a few. We also need to convert the years into a four-digit number as a two-digit year encoding is only common on legacy data storage systems.

In [ ]:
insurance_data = insurance_data[insurance_data["EFFECTIVE_YR"].astype(str).str.match(r"^\d{2}$")]

def convert_year(y):
    y = int(y)
    if y > 18:
        return 1900 + y
    else:
        return 2000 + y

insurance_data["EFFECTIVE_YR_FULL"] = insurance_data["EFFECTIVE_YR"].apply(convert_year)

insurance_data.shape

After cleaning the effective year column, we have lost 1171 rows.

In [ ]:
year_counts = insurance_data["EFFECTIVE_YR_FULL"].value_counts().sort_index()

year_counts = year_counts[
    (year_counts.index >= 1992) &
    (year_counts.index <= 2018)
]

year_counts.plot(kind='bar')
plt.xlabel("Effective Year")
plt.ylabel("Policy Count")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
year_counts = insurance_data["EFFECTIVE_YR_FULL"].value_counts().sort_index()
print(year_counts)

The data shows that most of the insured vehicles were first insured in 2011 and later. After contacting the postdoctoral fellow that published the dataset, I was informed that the meaning of this variable is not very well documented. There are values of 1947 and prior, eventhough, in Ethiopia, the first motor insurance was issued in 1947. This variable will be dropped.

In [ ]:
insurance_data.drop(columns=["EFFECTIVE_YR", "EFFECTIVE_YR_FULL"], inplace=True)

### Insurance type

In [ ]:
insurance_data["INSR_TYPE"].value_counts()

There are a total of three types of insurance: 1201 - private, 1202 - commercial and 1204 - motor trade road risk (for motor trade workers that drive vehicles they do not personally own, such as mechanics when testing repaired vehicles).

Let's change the values so they make more sense.

In [ ]:
insurance_type_mapping = {1202:'PRIVATE', 1201:'COMMERCIAL', 1204:'MOTOR TRADE'}

insurance_data["INSR_TYPE"] = insurance_data["INSR_TYPE"].map(insurance_type_mapping)

In [ ]:
insurance_data["INSR_TYPE"].value_counts()

### Insured value

In [ ]:
insurance_data["INSURED_VALUE"].value_counts()

343235 vehicles in the dataset have no provided insurance value. Insured value of 0 means the policyholder has the liability insurance coverage only, not the comprehensive coverage while insured value higher than 0 indicates comprehensive coverage.

The difference between the liability and comprehensive insurance is that liability insurance covers damage or injury you cause to other people or their property (and not repairs to your vehicle) and comprehensive insurance covers non-collision damage to your own car.

For the purpose of developing a model, we may create an additional variable that indicates the type of insurance the policyholder has.

In [ ]:
insurance_data["INSR_COVER"] = np.where(
    insurance_data["INSURED_VALUE"] == 0,
    "liability",
    "comprehensive"
)

insurance_data["INSR_COVER"].value_counts()

In [ ]:
insurance_data["INSURED_VALUE"].describe()

### Object ID

In [ ]:
insurance_data["OBJECT_ID"].value_counts()

Some vehicles appear more that 7 times which is not possible.

In [ ]:
insurance_data[insurance_data["OBJECT_ID"].astype(str) == "5000116673"]

In [ ]:
(insurance_data["OBJECT_ID"].value_counts() > 7).sum()

There are a total of 7269 OBJECT_IDs that apprear more than 7 times. This issue should be with the parties responsible with data collection. Nevertheless, correcting each one would have to be done manually by hand and therefore it would be much simpler just to remove the entries completely. In a real-world scenario, irregular values should be investigated.

In [ ]:
object_id_counts = insurance_data["OBJECT_ID"].value_counts()

ids_to_remove = object_id_counts[object_id_counts > 7].index

insurance_data = insurance_data[~insurance_data["OBJECT_ID"].isin(ids_to_remove)]

print(insurance_data.shape)

### Year of production

In [ ]:
insurance_data["PROD_YEAR"].value_counts()

In [ ]:
insurance_data["PROD_YEAR"].describe()

In [ ]:
insurance_data["PROD_YEAR"].plot(kind='hist', bins=40)

There are no abnormalities in this attribute.

### Number of seats

In [ ]:
insurance_data["SEATS_NUM"].describe()

In [ ]:
insurance_data["SEATS_NUM"].value_counts()

There are a total of 53866 vehicles with 0 seats which is impossible. Also, the number of seats should not exceed 256 (seats in the largest bus in the world). Other values will be considered correct.

In [ ]:
insurance_data = insurance_data[(insurance_data["SEATS_NUM"] > 0) & (insurance_data["SEATS_NUM"] <= 256)]

print(insurance_data.shape)

### Carrying capacity

In [ ]:
insurance_data["CARRYING_CAPACITY"].value_counts()

It is clear that the seat number variable and the carrying capacity variable are not clearly differentiated. They have been mixed up and should be removed.

In [ ]:
insurance_data.drop(columns=["SEATS_NUM", "CARRYING_CAPACITY"], inplace=True)

### Vehicle type

In [ ]:
insurance_data["TYPE_VEHICLE"].value_counts()

Nothing out of the ordinary here.

### Vehicle Weight

In [ ]:
insurance_data["CCM_TON"].describe()

In [ ]:
insurance_data["CCM_TON"].value_counts()

This variable is related to the weight of the vehicle. Since it is not clearly described how and what units are being used, we will remove it.

In [ ]:
insurance_data.drop(columns=["CCM_TON"], inplace=True)

### Vehicle maker

In [ ]:
insurance_data["MAKE"].value_counts()

In [ ]:
insurance_data["MAKE"].value_counts()[insurance_data["MAKE"].value_counts() <= 200]

There is a large number of vehicle maker names that are not representative of the manufacturer or are miss-spellings of the brand name, with 'TOYOTA MERCHEDIS' being the most humorous one. Since there is a total number of 657 unique brands, individual corrections would be too cumbersome. Instead, we can use a list of car names and check for matches in the dataset.

We will scrape a complete list of car brands from a car brand [website](https://www.carlogos.org/).

In [ ]:
url = "https://www.carlogos.org/car-brands-a-z/"

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/115.0.0.0 Safari/537.36"
    )
}

response = requests.get(url, headers=headers)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

brands = []

for dd in soup.find_all("dd"):
    a_tag = dd.find("a")
    if a_tag and a_tag.text.strip():
        brands.append(a_tag.text.strip())

brands = list(dict.fromkeys(brands))

print(brands)

len(brands)

The website states that there are 383 car brands on the website. The beginning of the list contains countries and the end contains other unrelated text. That should prove easy to clean.

In [ ]:
brands_clean = brands[21:-8]

print(brands_clean)

len(brands_clean)

We can now look for matching brands in our dataset. For that we will use the fuzzywuzzy library and search for matches using the Levenshtein distance.

In [ ]:
def match_brand(brand):
    match, score = process.extractOne(brand, brands_clean)
    print(match)
    return match if score >= 85 else 'UNKNOWN'

insurance_data["MANUFACTURER"] = insurance_data["MAKE"].apply(match_brand)

insurance_data["MANUFACTURER"].unique()

### Primary function of vehicle

In [ ]:
insurance_data["USAGE"].value_counts()

All functions seem valid.

### Paid claim sum

In [ ]:
insurance_data["CLAIM_PAID"] = insurance_data["CLAIM_PAID"].replace(np.nan, 0)

pd.set_option('display.float_format', '{:.2f}'.format)

print(insurance_data["CLAIM_PAID"].describe())

In [ ]:
insurance_data["CLAIM_PAID"].median()

In [ ]:
len(insurance_data[insurance_data["CLAIM_PAID"] == 0])

The payout values seem realistic since they are in Ethiopian birr. The largest paid sum is around 1,100,000 USD and the median no payout at all (0 USD).

### Insurance premium

The premium amounts are provided in Ethiopian birr (1000 Birr = 7,3 USD).

In [ ]:
insurance_data["PREMIUM"][insurance_data["PREMIUM"] <= 0].count()

There are 29 vehicles with a premium of 0 or less. This is not acceptable and we will remove them. I was informed by the publisher of the dataset that the company follows a principle of "No premium, no insurance". This method of record keeping is ineffective.

In [ ]:
insurance_data = insurance_data[insurance_data["PREMIUM"] > 0]

insurance_data["PREMIUM"].describe()

From the variable description it becomes clear that it is nowhere close to normality.

In [ ]:
params = expon.fit(insurance_data["PREMIUM"])

x = np.linspace(0, 50000, 1000)
pdf = expon.pdf(x, *params)

plt.hist(insurance_data["PREMIUM"], bins=500, density=True)
plt.plot(x, pdf, 'r')
plt.xlim([0, 50000])
plt.show()

The distribution closely resembles an exponential one.

In [ ]:
D, p_value = kstest(insurance_data["PREMIUM"], 'expon', args=params)

print(f"P-value: {p_value:.4f}")

As the p-value is less than 0.05, we fail to reject the null hypothesis. This distribution might as well exponential.

In [ ]:
log_premiums = np.log(insurance_data["PREMIUM"])

fig, axes = plt.subplots(2, 1, figsize=(8, 6), gridspec_kw={'height_ratios': [1, 4]})

axes[0].boxplot(log_premiums, vert=False, widths=0.6)
axes[0].set_xticks([])
axes[0].set_yticks([])

axes[1].hist(log_premiums, bins=22)
axes[1].set_xlabel('Log Premium')

plt.tight_layout(h_pad=0)

plt.show()

### Modelling

In [ ]:
insurance_data.info()

We are left with 12 variables that could be used for modelling: SEX, INSR_TYPE, INSURED_VALUE, PROD_YEAR, TYPE_VEHICLE, MAKE, USAGE, CLAIM_PAID, INSR_COVER, INSR_START_DAY, INSR_START_MONTH and INSR_START_YEAR.

In [ ]:
model = LinearRegression()

X = insurance_data[["INSR_TYPE", "INSURED_VALUE", "CLAIM_PAID", "INSR_COVER"]].copy()

X = pd.get_dummies(X, columns=["INSR_TYPE", "INSR_COVER"], drop_first=True)

Y = insurance_data["PREMIUM"]

X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2, random_state=87)

model.fit(X_train, Y_train)

Y_pred = model.predict(X_test)

r_sq = model.score(X_test, Y_test)

mae = sk.metrics.mean_absolute_error(Y_test, Y_pred)

r_sq, mae

In [ ]:
plt.scatter(Y_test, Y_pred, alpha=0.2)
plt.plot([Y_test.min(), Y_test.max()], [Y_test.min(), Y_test.max()], 'r')
plt.title("Accuracy Plot")
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()

In [ ]:
plt.scatter(Y_pred, Y_test - Y_pred, alpha=0.7)
plt.axhline(y=0, color='r')
plt.xlabel("Predicted")
plt.ylabel("Residuals")
plt.title("Residual Plot")
plt.show()

In [ ]:
correlation = np.corrcoef(Y_test, Y_pred)[0, 1]
print(f"Correlation: {correlation:.2f}")

In [ ]:
plt.hist(Y_test - Y_pred, bins=30, density=True, alpha=0.7)
plt.xlabel("Residuals")
plt.ylabel("Frequency")
plt.title("Distribution of Residuals")
plt.show()